In [ ]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"


# pkg/etl.py
import unittest

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.testing.utils import assertDataFrameEqual

# Create a SparkSession
spark = SparkSession.builder.appName("Sample PySpark ETL").getOrCreate()

sample_data = [{"name": "John    D.", "age": 30},
  {"name": "Alice   G.", "age": 25},
  {"name": "Bob  T.", "age": 35},
  {"name": "Eve   A.", "age": 28}]

df = spark.createDataFrame(sample_data)

# Define DataFrame transformation function
def remove_extra_spaces(df, column_name):
    # Remove extra spaces from the specified column using regexp_replace
    df_transformed = df.withColumn(column_name, f.regexp_replace(f.col(column_name), "\\s+", " "))

    return df_transformed

In [ ]:
# Define unit test base class
class PySparkTestCase(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.spark = SparkSession.builder.appName("Sample PySpark ETL").getOrCreate()

    @classmethod
    def tearDownClass(cls):
        cls.spark.stop()

# Define unit test
class TestTranformation(PySparkTestCase):
    def test_single_space(self):
        sample_data = [{"name": "John    D.", "age": 30},
                        {"name": "Alice   G.", "age": 25},
                        {"name": "Bob  T.", "age": 35},
                        {"name": "Eve   A.", "age": 28}]

        # Create a Spark DataFrame
        original_df = spark.createDataFrame(sample_data)

        # Apply the transformation function from before
        transformed_df = remove_extra_spaces(original_df, "name")

        expected_data = [{"name": "John D.", "age": 30},
        {"name": "Alice G.", "age": 25},
        {"name": "Bob T.", "age": 35},
        {"name": "Eve A.", "age": 28}]

        expected_df = spark.createDataFrame(expected_data)

        assertDataFrameEqual(transformed_df, expected_df)

In [4]:
unittest.main(argv=[''], verbosity=0, exit=False)

/opt/conda/lib/python3.11/socket.py:777: ResourceWarning: unclosed <socket.socket fd=60, family=2, type=1, proto=6, laddr=('127.0.0.1', 36250), raddr=('127.0.0.1', 44247)>
  self._sock = None
/opt/conda/lib/python3.11/socket.py:777: ResourceWarning: unclosed <socket.socket fd=60, family=2, type=1, proto=6, laddr=('127.0.0.1', 58960), raddr=('127.0.0.1', 33913)>
  self._sock = None
----------------------------------------------------------------------
Ran 1 test in 3.043s

OK


In [ ]:
%%sh
ls ./